In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2016-1-1 --end 2020-1-1 -o results.pickle

# Zipline trading algorithm
# Taken from zipline.examples.olmar

import numpy as np

from zipline.finance import commission, slippage

STOCKS = ['000001', '300001', '600000', '002024', '000002', '600645', '300002']


# On-Line Portfolio Moving Average Reversion

# More info can be found in the corresponding paper:
# http://icml.cc/2012/papers/168.pdf
def initialize(algo, eps=1, window_length=5):
    algo.stocks = STOCKS
    algo.sids = [algo.symbol(symbol) for symbol in algo.stocks]
    algo.m = len(algo.stocks)
    algo.price = {}
    algo.b_t = np.ones(algo.m) / algo.m
    algo.eps = eps
    algo.window_length = window_length

    algo.set_commission(commission.PerShare(cost=0))
    algo.set_slippage(slippage.FixedSlippage(spread=0))


def handle_data(algo, data):
    m = algo.m

    x_tilde = np.zeros(m)
    b = np.zeros(m)

    # find relative moving average price for each asset
    mavgs = data.history(algo.sids, 'price', algo.window_length, '1d').mean()
    for i, sid in enumerate(algo.sids):
        price = data.current(sid, "price")
        # Relative mean deviation
        x_tilde[i] = mavgs[sid] / price

    ###########################
    # Inside of OLMAR (algo 2)
    x_bar = x_tilde.mean()

    # market relative deviation
    mark_rel_dev = x_tilde - x_bar

    # Expected return with current portfolio
    exp_return = np.dot(algo.b_t, x_tilde)
    weight = algo.eps - exp_return
    variability = (np.linalg.norm(mark_rel_dev)) ** 2

    # test for divide-by-zero case
    if variability == 0.0:
        step_size = 0
    else:
        step_size = max(0, weight / variability)

    b = algo.b_t + step_size * mark_rel_dev
    b_norm = simplex_projection(b)
    np.testing.assert_almost_equal(b_norm.sum(), 1)

    rebalance_portfolio(algo, data, b_norm)

    # update portfolio
    algo.b_t = b_norm


def rebalance_portfolio(algo, data, desired_port):
    # rebalance portfolio
    for i, sid in enumerate(algo.sids):
        algo.order_target_percent(sid, desired_port[i])


def simplex_projection(v, b=1):
    """Projection vectors to the simplex domain
    Implemented according to the paper: Efficient projections onto the
    l1-ball for learning in high dimensions, John Duchi, et al. ICML 2008.
    Implementation Time: 2011 June 17 by Bin@libin AT pmail.ntu.edu.sg
    Optimization Problem: min_{w}\| w - v \|_{2}^{2}
    s.t. sum_{i=1}^{m}=z, w_{i}\geq 0
    Input: A vector v \in R^{m}, and a scalar z > 0 (default=1)
    Output: Projection vector w
    :Example:
    >>> proj = simplex_projection([.4 ,.3, -.4, .5])
    >>> print(proj)
    array([ 0.33333333, 0.23333333, 0. , 0.43333333])
    >>> print(proj.sum())
    1.0
    Original matlab implementation: John Duchi (jduchi@cs.berkeley.edu)
    Python-port: Copyright 2013 by Thomas Wiecki (thomas.wiecki@gmail.com).
    """

    v = np.asarray(v)
    p = len(v)

    # Sort v into u in descending order
    v = (v > 0) * v
    u = np.sort(v)[::-1]
    sv = np.cumsum(u)

    rho = np.where(u > (sv - b) / np.arange(1, p + 1))[0][-1]
    theta = np.max([0, (sv[rho] - b) / (rho + 1)])
    w = (v - theta)
    w[w < 0] = 0
    return w

,period_open,period_close,longs_count,shorts_count,long_value,short_value,pnl,capital_used,orders,long_exposure,...,algo_volatility,benchmark_period_return,benchmark_volatility,max_leverage,algorithm_period_return,alpha,beta,sharpe,sortino,max_drawdown
2016-01-04 07:01:00+00:00,2016-01-04 01:31:00+00:00,2016-01-04 07:01:00+00:00,0,0,0.00,0.0,0.00,0.00,"[{'id': '233b01d7343f45229053ab695bd4f454', 'd...",0.00,...,NaN,-0.070206,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000
2016-01-05 07:01:00+00:00,2016-01-05 01:31:00+00:00,2016-01-05 07:01:00+00:00,6,0,8433262.70,0.0,0.00,-8433262.70,"[{'id': '233b01d7343f45229053ab695bd4f454', 'd...",8433262.70,...,0.000000,-0.067603,0.819490,0.843326,0.000000,0.000000,0.000000,NaN,NaN,0.000000
2016-01-06 07:01:00+00:00,2016-01-06 01:31:00+00:00,2016-01-06 07:01:00+00:00,6,0,8789705.11,0.0,213692.15,-142750.26,"[{'id': '9a80858294424f88ae8197acb9a1dbf7', 'd...",8789705.11,...,0.195852,-0.051246,0.745908,0.860581,0.021369,10.885381,0.165332,9.165151,NaN,0.000000
2016-01-07 07:01:00+00:00,2016-01-07 01:31:00+00:00,2016-01-07 07:01:00+00:00,6,0,8065106.33,0.0,-694639.14,29959.64,"[{'id': 'c6d77ec49924433aae5f5a662957eb76', 'd...",8065106.33,...,0.617423,-0.117026,0.738897,0.860581,-0.048095,2.793592,0.569324,-4.759146,-5.443335,-0.068011
2016-01-08 07:01:00+00:00,2016-01-08 01:31:00+00:00,2016-01-08 07:01:00+00:00,6,0,8279347.83,0.0,117575.44,-96666.06,"[{'id': '7f69e45e9dfb47d99e31953ef74fdd87', 'd...",8279347.83,...,0.561220,-0.099020,0.732421,0.860581,-0.036337,1.712154,0.547807,-3.079372,-3.579344,-0.068011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25 07:01:00+00:00,2019-12-25 01:31:00+00:00,2019-12-25 07:01:00+00:00,3,0,50526719.79,0.0,242226.14,8066.70,"[{'id': 'f27e569073c94e20b44f180e96bbb531', 'd...",50526719.79,...,4.268365,0.069649,0.190885,80.011361,4.053105,-0.944905,0.852616,-0.668075,-0.668192,-7.346965
2019-12-26 07:01:00+00:00,2019-12-26 01:31:00+00:00,2019-12-26 07:01:00+00:00,3,0,50843319.44,0.0,316900.91,301.26,"[{'id': 'e9084a91a75c4cc495ca2e4e93920ed3', 'd...",50843319.44,...,4.266176,0.079062,0.190837,80.011361,4.084795,-0.944759,0.853245,-0.667349,-0.667467,-7.346965
2019-12-27 07:01:00+00:00,2019-12-27 01:31:00+00:00,2019-12-27 07:01:00+00:00,5,0,50907848.04,0.0,-225648.20,-290176.80,"[{'id': '83e9e81c5e454f3ba068af2e6660fca1', 'd...",50907848.04,...,4.263982,0.078000,0.190740,80.011361,4.062230,-0.944646,0.853182,-0.667276,-0.667394,-7.346965
2019-12-30 07:01:00+00:00,2019-12-30 01:31:00+00:00,2019-12-30 07:01:00+00:00,5,0,50876639.75,0.0,249046.12,280254.41,"[{'id': 'fe23d2f0a85d4bd1b3f40bdf8c22661c', 'd...",50876639.75,...,4.261799,0.093976,0.190788,80.011361,4.087135,-0.944593,0.853567,-0.666633,-0.666752,-7.346965
